## Importing

In [ ]:
import pandas as pd
import scipy as sp
import numpy as np

import emcee
from multiprocessing import Pool

import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
from matplotlib import font_manager
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import ListedColormap
from cycler import cycler
import corner
from typing import Any, Dict, List, Optional, Tuple, Union

import lymph

To do: 
Sampling mit ultimate truth consensus. 

## Creating or Loading Model

In [ ]:
filename = "../data/consensus_system.hdf5"

graph = {
    ('tumor', 'primary'): ['I', 'II', 'III', 'IV', 'V', 'VII'],
    ('lnl', 'I'): [],
    ('lnl', 'II'): ['I', 'III', 'V', 'VII'],
    ('lnl', 'III'): ['IV'],
    ('lnl', 'IV'): [],
    ('lnl', 'V'): [],
    ('lnl', 'VII'): [],
}
consensus_systm = lymph.Unilateral(graph=graph)


print(consensus_systm)

## Modalities

In [ ]:
spsn = {"consensus": [1., 1.]}
#                         ^   ^
#                specificty   sensitivity
consensus_systm.modalities = spsn

## Data

In [ ]:

def layer_involvement_statistical(inv: Optional[int], spsns: List[tuple])-> int:
    """Determine which layers are involved given the different (conflicting) measurements:

        Args:
            inv:    For one single layer, one entry gives the measured involvement for the
                    respective modality
            spsns:  Each entry of the list is the spsn tuple of the respective modality

    """
    p_healthy = 1
    p_involved = 1
    for i , spsn in enumerate(spsns):
        if(inv[i] == True):
            p_healthy *= 1-spsn[0]
            p_involved *= spsn[1]
        elif(inv[i] == False):
            p_healthy *= spsn[0]
            p_involved *= 1-spsn[1]
    if(p_healthy < p_involved):
        return 1
    else:
        return 0

In [ ]:
ipsi_data = pd.read_csv("../data/ipsi_data.csv", header=[0,1] )

layer_cols = ipsi_data.columns.get_level_values(1)

#Delete Duplicates and t-stage from list
layers = list(dict.fromkeys(layer_cols))[:-1]


#Create Empty DataFrame to store the occurrences
occurr_table = pd.DataFrame(np.zeros((len(ipsi_data.index), len(layers))), columns=layers)

#Fill the occurrence table
for layer in layers:
    select = layer_cols.isin([layer])
    level_data = ipsi_data.loc[:, select].reset_index(drop=True)
    for index, row in level_data.iterrows():
        involved = layer_involvement_statistical(row.values, [(0.63, 0.81), (0.86, 0.79), (0.63, 0.81), (1., 1.)])
        if(involved):
            occurr_table.loc[occurr_table.index[index], layer] = 1


headers = [("consensus", layer) for layer in layers]
occurr_table.columns= pd.MultiIndex.from_tuples(headers)
occurr_table[("info", "t_stage")]= ipsi_data[("info", "t_stage")]
print(occurr_table)

occurr_table.to_csv("../data/consensus.csv", index=None)

consensus_systm.patient_data=occurr_table

## Storage of model

In [ ]:
consensus_systm.to_hdf(
    filename=filename, 
    name="extended/model"
)

## Likelihood

In [ ]:
#settings for the binom distributions
early_p=0.3
max_t=10
t = np.arange(max_t + 1)

def llh(theta):
    spread_probs, late_p = theta[:-1], theta[-1]
    
    print("run")
    if late_p > 1. or late_p < 0.:
        return -np.inf
    
    
    time_dists={
        "early": sp.stats.binom.pmf(t, max_t, early_p),
        "late" : sp.stats.binom.pmf(t, max_t, late_p),
    }
     
    return consensus_systm.marginal_log_likelihood(spread_probs, t_stages=["early", "late"], time_dists=time_dists)

## Sampling

In [43]:
# Settings for the sampler
ndim = len(consensus_systm.spread_probs) + 1
nwalkers = 20 * ndim
nstep = 10000
burnin = 5000
moves = [(emcee.moves.DEMove(), 0.8), (emcee.moves.DESnookerMove(), 0.2)]


#prepare the backend
new_backend = emcee.backends.HDFBackend(
    filename=filename,
    name="extended/samples"
)
#new_backend.reset(nwalkers, ndim)
print("Initial size: {0}".format(new_backend.iteration))

# starting point
initial_spread_probs = np.random.uniform(low=0., high=1., size=(nwalkers,ndim))


if __name__ == "__main__":
    sampler = emcee.EnsembleSampler(
        nwalkers, ndim, 
        llh, 
        moves=moves,
        backend=new_backend
    )
    sampler.run_mcmc(initial_spread_probs, nstep-new_backend.iteration, progress=True)
    samples_HMM = sampler.get_chain(flat=True, discard=burnin)
    print(samples_HMM)      





Initial size: 7081
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run


  0%|          | 0/2919 [00:00<?, ?it/s]

run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run


  0%|          | 1/2919 [00:02<1:52:03,  2.30s/it]

run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run


...

  0%|          | 11/2919 [00:12<50:49,  1.05s/it]

run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run


Traceback (most recent call last):
  File "C:\Users\Luca\anaconda3\envs\lymph\lib\site-packages\emcee\ensemble.py", line 624, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "C:\Users\Luca\AppData\Local\Temp\ipykernel_15404\2811146381.py", line 19, in llh
    return consensus_systm.marginal_log_likelihood(spread_probs, t_stages=["early", "late"], time_dists=time_dists)
  File "C:\Users\Luca\anaconda3\envs\lymph\lib\site-packages\lymph\unilateral.py", line 866, in marginal_log_likelihood
    return self.log_likelihood(
  File "C:\Users\Luca\anaconda3\envs\lymph\lib\site-packages\lymph\unilateral.py", line 807, in log_likelihood
    if(np.all(self._evolve(t_last=max_t)==0)):
  File "C:\Users\Luca\anaconda3\envs\lymph\lib\site-packages\lymph\unilateral.py", line 699, in _evolve
    state_probs[i] = state
KeyboardInterrupt
  0%|          | 11/2919 [00:12<56:34,  1.17s/it]

run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
run
emcee: Exception while calling your likelihood function:
  params: [0.1420258  0.19937567 0.85496872 0.58871177 0.18549484 0.64331776
 0.22293914 0.68926383 0.87030537 0.82959171 0.31948493 0.08572591]
  args: []
  kwargs: {}
  exception:


KeyboardInterrupt: 